In [28]:
import nltk
from nltk.corpus import treebank
from nltk import PCFG,CFG

cfg_grammar = CFG.fromstring('''
    S -> NP VP
    NP -> ART N | N N | N | NP PP
    VP -> V | V NP | V NP PP
    PP -> P NP
    ART -> 'a'
    N -> 'flower' | 'a' | 'blooms'
    V -> 'blooms' | 'flower'
''')

pcfg_grammar = PCFG.fromstring('''
    S -> NP VP [1.0]
    NP -> ART N [0.53] | N N [0.09] | N [0.14] | NP PP [0.24]
    VP -> V [0.386] | V NP [0.393] | V NP PP [0.22]
    PP -> P NP [1.0]
    ART -> 'a' [1.0]
    N -> 'flower' [0.8] | 'a' [0.1] | 'blooms' [0.1]
    V -> 'blooms' [0.8] | 'flower' [0.2]
''')

In [30]:
cfg_grammar.productions()

[S -> NP VP,
 NP -> ART N,
 NP -> N N,
 NP -> N,
 NP -> NP PP,
 VP -> V,
 VP -> V NP,
 VP -> V NP PP,
 PP -> P NP,
 ART -> 'a',
 N -> 'flower',
 N -> 'a',
 N -> 'blooms',
 V -> 'blooms',
 V -> 'flower']

In [134]:
pcfg_grammar.productions()

[S -> NP VP [1.0],
 NP -> ART N [0.53],
 NP -> N N [0.09],
 NP -> N [0.14],
 NP -> NP PP [0.24],
 VP -> V [0.386],
 VP -> V NP [0.393],
 VP -> V NP PP [0.22],
 PP -> P NP [1.0],
 ART -> 'a' [1.0],
 N -> 'flower' [0.8],
 N -> 'a' [0.1],
 N -> 'blooms' [0.1],
 V -> 'blooms' [0.8],
 V -> 'flower' [0.2]]

# Answer 1

In [135]:
words = ['a','flower','blooms']

chart_parser = nltk.ChartParser(cfg_grammar)
trees = []
for tree in chart_parser.parse(words):
    trees.append(tree)
    tree.pretty_print()
    
    

          S          
      ____|______     
     NP          VP  
  ___|____       |    
ART       N      V   
 |        |      |    
 a      flower blooms

          S          
      ____|______     
     NP          VP  
  ___|____       |    
 N        N      V   
 |        |      |    
 a      flower blooms

      S              
  ____|_____          
 |          VP       
 |     _____|____     
 NP   |          NP  
 |    |          |    
 N    V          N   
 |    |          |    
 a  flower     blooms



# Answer 2

In [136]:
viterbi_parser = nltk.ViterbiParser(pcfg_grammar)
for tree in viterbi_parser.parse(words):
    print tree.pretty_print()

          S          
      ____|______     
     NP          VP  
  ___|____       |    
ART       N      V   
 |        |      |    
 a      flower blooms

None


# Answer 3

In [137]:
import re

production_reg_exp = re.compile('^\w+ -> \'*[\w \w]*\'*')
prob_reg_exp = re.compile('\[\d.\d+\]$')

prob_lookup_table = {}
for production in pcfg_grammar.productions():
    try:
        prob_lookup_table[production_reg_exp.findall(str(production))[0].strip()] = \
        float(prob_reg_exp.findall(str(production))[0].replace('[','').replace(']',''))
    except KeyError:
        print 'Key not found'
        
print prob_lookup_table

{"N -> 'blooms'": 0.1, 'VP -> V NP PP': 0.22, 'NP -> NP PP': 0.24, "ART -> 'a'": 1.0, 'NP -> N': 0.14, 'NP -> ART N': 0.53, "N -> 'flower'": 0.8, 'NP -> N N': 0.09, "N -> 'a'": 0.1, "V -> 'flower'": 0.2, 'VP -> V': 0.386, 'VP -> V NP': 0.393, 'S -> NP VP': 1.0, "V -> 'blooms'": 0.8, 'PP -> P NP': 1.0}


In [138]:
productions,tree_prob,c = {},{},1
for tree in trees:
    productions['tree'+str(c%4)] = tree.productions()
    c += 1
    
for tree,tree_production in productions.items():
    for production in tree_production:
        try:
            tree_prob[tree] *= prob_lookup_table[str(production)]
        except KeyError:
            tree_prob[tree] = 1

print tree_prob

sum_prob = 0.0
for prob in tree_prob.values():
    sum_prob += prob
    
print 'total probability:',sum_prob

{'tree1': 0.13093120000000003, 'tree3': 1.5405600000000004e-05, 'tree2': 0.00222336}
total probability: 0.1331699656
